In [ ]:
%load_ext autoreload

# 数据准备

In [ ]:
%autoreload
import sys,os
import numpy as np
import torch
import matplotlib.pyplot as plt
from tianmoucv.isp import SD2XY
from tianmoucv.data import TianmoucDataReader
import torch.nn.functional as F
import cv2

train='/data/lyh/tianmoucData/tianmoucReconDataset/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]
val='/data/lyh/tianmoucData/tianmoucReconDataset/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]
key_list = []
print('---------------------------------------------------')
for sampleset in traindata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
print('---------------------------------------------------')
for sampleset in valdata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
        
all_data = valdata + traindata

## (1) 镜像SD数据

In [ ]:
%autoreload
import torch
import torch.nn.functional as F
import math
from tianmoucv.isp import flip_xyd
from tianmoucv.isp import vizDiff

aim = 4 # 读取的目标COP帧的帧号
N = 1   # read continue N frames （连续读取N帧COP以及对应的AOP）

# 这些形式都可以读取数据
datapath = "/data/lyh/tianmoucData/tianmoucReconDataset/train/normal/train_cross2/"
datapath = [train,val]
datapath = valdata + traindata
key_list = ['train_cross2']

for key in key_list:
    dataset = TianmoucDataReader(datapath,N=N,matchkey=key,camera_idx=0,_correct_sd_upsampler=True)
    img_list = []
    for index in range(aim,min(aim+1,len(dataset))):
        sample = dataset[index]

        #[C,T,H,W] -> [T,C,H,W]
        txydiff = torch.Tensor(sample['txydiff']).permute(1,0,2,3)
        length = txydiff.shape[0]    
        dataRatio = sample['dataRatio']
    
        for dims in [[2],[3],[2,3]]:
            print('翻转的维度：',dims)
            plt.figure(figsize=(12,2*N))  
            for i in range(N):
                F = sample['F'+str(i)].numpy() #第i帧COP
                F_HDR = sample['F'+str(i)+'_HDR'].numpy() #第i帧的HDR合成结果
                # 把time和batch维度合并到第0维输入
                flipped_xyd =flip_xyd(txydiff[:,1:,...],dims=dims) 
                
                F_HDR[F_HDR>1]=1
                F_HDR[F_HDR<0]=0
                flipped_xyd = flipped_xyd[0,:,...] #第i帧到第i+1帧COP之间的AOP数据
                flipped_xyd = torch.cat([flipped_xyd,torch.zeros([1,320,640])],dim=0)

                plt.subplot(N,3,1+i*3)
                plt.imshow(F)
                plt.subplot(N,3,2+i*3)
                plt.imshow(flipped_xyd.permute(1,2,0)*2) #可视化第0帧，乘个Gain方便看
                plt.subplot(N,3,3+i*3)
                plt.imshow(F_HDR)
            plt.show()
    break

## (2) 旋转SD数据

In [ ]:
%autoreload
import torch
import torch.nn.functional as F
import math
from tianmoucv.isp import rotate_txyd,rotate_nd_tensor

aim = 4 # 读取的目标COP帧的帧号
N = 1   # read continue N frames （连续读取N帧COP以及对应的AOP）

# 这些形式都可以读取数据
datapath = "/data/lyh/tianmoucData/tianmoucReconDataset/train/normal/train_cross2/"
datapath = [train,val]
datapath = valdata + traindata
key_list = ['train_cross2']

for key in key_list:
    dataset = TianmoucDataReader(datapath,N=N,matchkey=key,camera_idx=0,_correct_sd_upsampler=True)
    img_list = []
    for index in range(aim,min(aim+1,len(dataset))):
        sample = dataset[index]
        txydiff = torch.Tensor(sample['txydiff']).permute(1,0,2,3)
        length = txydiff.shape[1]
        dataRatio = sample['dataRatio']
        
        for deg in [0,45,90,135,180,225,270]:
            print('旋转：',deg, '度')
            plt.figure(figsize=(12,2*N))  
            for i in range(N):
                Fi = sample['F'+str(i)].numpy() #第i帧COP
                F_HDR = sample['F'+str(i)+'_HDR'].numpy() #第i帧的HDR合成结果
                F_HDR = rotate_nd_tensor(torch.FloatTensor(F_HDR).permute(2,0,1),deg).permute(1,2,0)
                Fi = rotate_nd_tensor(torch.FloatTensor(Fi).permute(2,0,1),deg).permute(1,2,0)
                # 把time和batch维度合并到第0维输入
                rotate_txydiff =rotate_txyd(txydiff,deg).permute(1,0,2,3) 
                
                F_HDR[F_HDR>1]=1
                F_HDR[F_HDR<0]=0
                Ix_r = rotate_txydiff[1,0,...]
                Iy_r = rotate_txydiff[2,0,...]

                axs = plt.subplot(N,4,1+i*3)
                plt.imshow(Fi)
                axs = plt.subplot(N,4,2+i*3)
                plt.imshow(Ix_r**2) #可视化第0帧，乘个Gain方便看
                axs.set_title('Ix')
                axs = plt.subplot(N,4,3+i*3)
                plt.imshow(Iy_r**2) #可视化第0帧，乘个Gain方便看
                axs.set_title('Iy')
                axs = plt.subplot(N,4,4+i*3)
                plt.imshow(F_HDR)
            plt.show()
    break

## 测试旋转后SD和RGB合成效果的一致性

In [ ]:
%autoreload
import torch.nn as nn
import math
import time
from tianmoucv.isp import rotate_nd_tensor,rotate_xyd
from tianmoucv.isp import SD2XY
from tianmoucv.proc.reconstruct import poisson_blending
from tianmoucv.data import TianmoucDataReader
import torch.nn.functional as F

psnrcount =0
count = 0
key_list = ['test_tunnel7_hdr_ae']

for key in key_list:
    dataset = TianmoucDataReader(all_data,MAXLEN=500,matchkey=key)
    dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1,\
                                          num_workers=4, pin_memory=False, drop_last = False)
    PSNR = 0
    img_list = []
    for index,sample in enumerate(dataLoader,0):
        if index<= 20:
            continue
        for deg in [0,45,90,135,180]:
            F0 = sample['F0'][0,...]
            F_HDR = sample['F0_HDR'][0,...]
            raw_F0 = F0.clone()
            #只有第0帧可以合成
            for t in [0]:
                tsdiff = sample['rawDiff'][0,...].clone()/128.0        
                SD = tsdiff[1:,t,...].permute(1,2,0)
                Ix,Iy= SD2XY(SD)
                Ix = F.interpolate(torch.Tensor(Ix).unsqueeze(0).unsqueeze(0), size=(320,640), mode='bilinear').squeeze(0).squeeze(0)
                Iy = F.interpolate(torch.Tensor(Iy).unsqueeze(0).unsqueeze(0), size=(320,640), mode='bilinear').squeeze(0).squeeze(0)
                rotated_F0 = rotate_nd_tensor(F0.permute(2,0,1),angle=deg).permute(1,2,0)
                rotated_xy = rotate_xyd(torch.stack([Ix,Iy],dim=0),angle=deg)
                Ix_r = rotated_xy[0,...]
                Iy_r = rotated_xy[1,...]
                blend_hdr = poisson_blending(Ix_r,Iy_r, srcimg= rotated_F0,iteration=20, mask_rgb=True,mask_th=36)

            plt.figure(figsize=(12,6))
            plt.subplot(1,3,1)  
            plt.imshow(blend_hdr)
            plt.subplot(1,3,2)  
            plt.imshow(Ix_r**2)
            plt.subplot(1,3,3)  
            plt.imshow(Iy_r**2)
            plt.show()
        break